In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 

In [2]:
pwd = os.getcwd() # Current Directory

In [3]:
df_raw = pd.read_csv(pwd+"/Perovskite_dataset/Perovskite_database_content_all_data.csv",low_memory=False) # Loading Perovskite Database
df_raw.shape # Shape of the dataset

(42443, 410)

In [4]:
df_raw.columns.to_list() # List of columns

['Ref_ID',
 'Ref_ID_temp',
 'Ref_name_of_person_entering_the_data',
 'Ref_data_entered_by_author',
 'Ref_DOI_number',
 'Ref_lead_author',
 'Ref_publication_date',
 'Ref_journal',
 'Ref_part_of_initial_dataset',
 'Ref_original_filename_data_upload',
 'Ref_free_text_comment',
 'Ref_internal_sample_id',
 'Cell_stack_sequence',
 'Cell_area_total',
 'Cell_area_measured',
 'Cell_number_of_cells_per_substrate',
 'Cell_architecture',
 'Cell_flexible',
 'Cell_flexible_min_bending_radius',
 'Cell_semitransparent',
 'Cell_semitransparent_AVT',
 'Cell_semitransparent_wavelength_range',
 'Cell_semitransparent_raw_data',
 'Module',
 'Module_number_of_cells_in_module',
 'Module_area_total',
 'Module_area_effective',
 'Module_JV_data_recalculated_per_cell',
 'Substrate_stack_sequence',
 'Substrate_thickness',
 'Substrate_area',
 'Substrate_supplier',
 'Substrate_brand_name',
 'Substrate_deposition_procedure',
 'Substrate_surface_roughness_rms',
 'Substrate_etching_procedure',
 'Substrate_cleaning_proc

In [5]:
# Initial selection of columns
col_lst = [
       'Ref_DOI_number', 
       #'Cell_stack_sequence', 
       'Cell_architecture',
       'Cell_flexible', 'Cell_semitransparent', 
       'Substrate_stack_sequence',
       #'Substrate_thickness',  ** Substrate_thickness contains more than 42000 missing data** 
       'ETL_stack_sequence', 
       'ETL_thickness',
       'ETL_deposition_procedure', 
       #'Perovskite_dimension_0D',
       'Perovskite_dimension_2D', 'Perovskite_dimension_2D3D_mixture',
       'Perovskite_dimension_3D',
       'Perovskite_dimension_3D_with_2D_capping_layer',
       'Perovskite_composition_perovskite_ABC3_structure',
       #'Perovskite_composition_short_form', 
       'Perovskite_composition_long_form',
       'Perovskite_thickness', 'Perovskite_composition_inorganic',
       #'Perovskite_composition_leadfree', 
       'Perovskite_band_gap',
       'Perovskite_band_gap_graded',
       #'Perovskite_deposition_number_of_deposition_steps',
       'Perovskite_deposition_procedure',
       #'Perovskite_deposition_synthesis_atmosphere',
       'Perovskite_deposition_solvents',
       'Perovskite_deposition_quenching_induced_crystallisation',
       #'Perovskite_deposition_quenching_media',
       'Perovskite_deposition_thermal_annealing_temperature',
       'Perovskite_deposition_thermal_annealing_time',
       'Perovskite_deposition_solvent_annealing', 
       'HTL_stack_sequence',
       'HTL_thickness_list', 'HTL_deposition_procedure',
       'Backcontact_stack_sequence', 
       'Backcontact_thickness_list',
       'Backcontact_deposition_procedure', 
       'JV_default_Voc', 
       'JV_default_Jsc',
       'JV_default_FF', 
       'JV_default_PCE']

In [6]:
df_selected = df_raw[col_lst] # Selecting columns from the raw dataset
df_selected.shape # Shape of the subset

(42443, 34)

In [7]:
df_selected.isnull().sum() # Checking no. of missing values on each columns

Ref_DOI_number                                                 0
Cell_architecture                                              0
Cell_flexible                                                  0
Cell_semitransparent                                           0
Substrate_stack_sequence                                       0
ETL_stack_sequence                                             0
ETL_thickness                                              23913
ETL_deposition_procedure                                       0
Perovskite_dimension_2D                                        0
Perovskite_dimension_2D3D_mixture                              0
Perovskite_dimension_3D                                        0
Perovskite_dimension_3D_with_2D_capping_layer                  0
Perovskite_composition_perovskite_ABC3_structure               0
Perovskite_composition_long_form                               0
Perovskite_thickness                                       29126
Perovskite_composition_in

Removing missing values in target variable columns

In [8]:
df_new = df_selected.dropna(subset=['JV_default_Voc','JV_default_Jsc','JV_default_FF','JV_default_PCE'])
df_new.shape 

(40073, 34)

In [9]:
df_new.dtypes # Checking dataypes of each columns

Ref_DOI_number                                              object
Cell_architecture                                           object
Cell_flexible                                                 bool
Cell_semitransparent                                          bool
Substrate_stack_sequence                                    object
ETL_stack_sequence                                          object
ETL_thickness                                               object
ETL_deposition_procedure                                    object
Perovskite_dimension_2D                                       bool
Perovskite_dimension_2D3D_mixture                             bool
Perovskite_dimension_3D                                       bool
Perovskite_dimension_3D_with_2D_capping_layer                 bool
Perovskite_composition_perovskite_ABC3_structure              bool
Perovskite_composition_long_form                            object
Perovskite_thickness                                        ob

Removing Unknown or none values

In [10]:
def finder(df:pd.DataFrame,string:str):
    """
    This is an helper function which
    1. Counts the frequency of particular string on each column
    2. Finds out the index no. where the string is present
    Input: 
    df -> Dataframe where finder function will run
    string -> String to find
    Returns: A dataframe with column names, counts and index no. 
    
    """
    df = df.astype(str)
    cols = df.columns
    col_lst = []
    count_lst = []
    idx_lst = []
    for col in cols:
        cond = df[col].str.contains(string)
        i = df[cond].index
        count = sum(cond)
        if count:
            col_lst.append(col)
            count_lst.append(count)
            idx_lst.append(i.to_list())
    out = pd.DataFrame({"Column_Name":col_lst,string+" count":count_lst,"Index":idx_lst})
    return out

def gen_drop_lst(lst):
    """ Converts Nested List to single list with all the values"""
    new_lst = []
    [new_lst.extend(x) for x in lst]
    return [*set(new_lst)] # removing duplicates

In [11]:
st = "Unknown|unkown|unknown|None|none" # strings seperated by '|' are in the dataset but needs to be removed
uk_counts = finder(df_new,st) # Finding the index 
uk_counts

,Column_Name,Unknown|unkown|unknown|None|none count,Index
0,Cell_architecture,5,"[13255, 13256, 13257, 13258, 40998]"
1,Substrate_stack_sequence,22,"[1150, 3789, 3790, 13255, 13256, 13257, 13258,..."
2,ETL_stack_sequence,308,"[562, 782, 1079, 1361, 1370, 1445, 2090, 2852,..."
3,ETL_thickness,4,"[37944, 37945, 37946, 37947]"
4,ETL_deposition_procedure,776,"[48, 323, 407, 408, 409, 410, 411, 412, 426, 4..."
5,Perovskite_deposition_procedure,140,"[426, 427, 428, 1655, 1656, 2863, 2864, 2865, ..."
6,Perovskite_deposition_solvents,3046,"[33, 34, 35, 36, 37, 38, 150, 151, 152, 153, 1..."
7,Perovskite_deposition_thermal_annealing_temper...,7889,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 48, 61, 6..."
8,Perovskite_deposition_thermal_annealing_time,8932,"[19, 20, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4..."
9,HTL_stack_sequence,2547,"[12, 13, 20, 39, 153, 171, 172, 173, 174, 175,..."


In [12]:
# Finding the row index where cell equals to "Unknown/none" and droping the entire row. 
r_drop = finder(df_new,st)['Index'].to_list()
r_drop_lst = gen_drop_lst(r_drop)
df_un_removed = df_new.drop(r_drop_lst) # Droping cells which contains unknown/none values
df_un_removed.shape

(27200, 34)

In [13]:
df_PCE = df_un_removed[df_un_removed['JV_default_PCE'] <= 25 ] # Getting the data which are below 25% PCE. All others are considered outliers
del_idxs = df_PCE.loc[df_PCE.HTL_thickness_list == "TRUE"].index # Index of some abnormal cells
df_no_outlier = df_PCE.drop(del_idxs) # Droping all outliers
df_no_outlier.shape # Shape of the dataset after droping ouliers

(27175, 34)

In [14]:
# columns to convert in float64 type
t_cols = ['ETL_thickness','Perovskite_thickness','Perovskite_band_gap','HTL_thickness_list','Backcontact_thickness_list']

#### Object columns conversion to Float64

In [15]:
def convert_to_num(val):
    """
    Some thickness data have individual layer thickness seperated by '|' which are considered as string type data.
    This function takes the sum of each layer and converts it into float type data. 
    """
    if isinstance(val, str):
        # split the string by '|'
        nums = val.split('|')
        # convert each element to float and sum them up
        total = sum([float(num) if num != 'nan' else 0 for num in nums])
        # return empty string if total is 0, else return the total
        return np.nan if total == 0 else total
    # if the value is not a string, return the original value
    return val

In [16]:
df_converted = df_no_outlier.copy()
for c in t_cols:
    df_converted[c] = df_converted[c].apply(convert_to_num) # converting the columns into float type data

In [19]:
df_converted.isnull().sum() # Checking for missing values in columns

Ref_DOI_number                                                 0
Cell_architecture                                              0
Cell_flexible                                                  0
Cell_semitransparent                                           0
Substrate_stack_sequence                                       0
ETL_stack_sequence                                             0
ETL_thickness                                              19104
ETL_deposition_procedure                                       0
Perovskite_dimension_2D                                        0
Perovskite_dimension_2D3D_mixture                              0
Perovskite_dimension_3D                                        0
Perovskite_dimension_3D_with_2D_capping_layer                  0
Perovskite_composition_perovskite_ABC3_structure               0
Perovskite_composition_long_form                               0
Perovskite_thickness                                       18735
Perovskite_composition_in

In [20]:
df_converted.duplicated().sum() # Checking for duplicate entries

718

In [21]:
df_cleaned = df_converted.drop_duplicates() # Droping the duplicate entries
df_cleaned.shape

(26457, 34)

In [22]:
df_cleaned.to_csv(pwd+"/Perovskite_dataset/perovskite_cleaned_dataset.csv",index=False) # Saving the cleaned dataset